In [13]:
# import library
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt 

In [14]:
# load the data
df = pd.read_csv("cpns_2019.csv")

In [15]:
# check total rows by instansi dan jabatan
df[['insNm','jabNm']].nunique()

insNm     522
jabNm    2201
dtype: int64

In [16]:
# grouping by jenis formasi
aggregat = {
    'jumlahFormasi':sum,
    'jumlahSubmit':sum,
    'jabNm':'count'
}
df.groupby('jenisFormasiNm').agg(aggregat)

,jumlahFormasi,jumlahSubmit,jabNm
jenisFormasiNm,,,
DIASPORA,9,5,8
DIASPORA (DAPAT DIISI DISABILITAS),93,12,3
LULUSAN TERBAIK,3581,19120,2191
LULUSAN TERBAIK (DAPAT DIISI DISABILITAS),732,4059,363
PENYANDANG DISABILITAS,2580,1865,2285
PUTRA/PUTRI PAPUA DAN PAPUA BARAT,398,3724,133
PUTRA/PUTRI PAPUA DAN PAPUA BARAT (DAPAT DIISI DISABILITAS),53,203,32
TENAGA CYBER,626,2156,4
UMUM,120122,3590323,89494


### **Menambahkan Group Instansi**

In [17]:
row_list = []
for i,j in df.iterrows():
    y = re.split("\s",j['insNm'])
    row_list.append(y[0])

tmp = pd.DataFrame(row_list, columns=['grp'])
tmp.groupby('grp')['grp'].count()

grp
Arsip              18
Badan            1625
Kejaksaan          41
Kementerian     12229
Kepolisian        175
Lembaga           212
Mahkamah           24
Ombudsman          86
Pemerintah      93636
Perpustakaan       37
Sekretariat        56
Setjen             55
Name: grp, dtype: int64

In [20]:
# searching like '%'
df[df['insNm'].str.contains('(?i)Setjen')]['insNm'].unique()

array(['Setjen Komisi Pemilihan Umum', 'Setjen KOMNAS HAM'], dtype=object)

In [22]:
# Mengubah group instansi "Badan" dan "Setjen" menjadi "Lembaga"
tmp['insGrp'] = tmp['grp'].apply(lambda x: 'Lembaga' if (x == 'Badan' or x=='Setjen') else x)
tmp.groupby('insGrp')['insGrp'].count()

insGrp
Arsip              18
Kejaksaan          41
Kementerian     12229
Kepolisian        175
Lembaga          1892
Mahkamah           24
Ombudsman          86
Pemerintah      93636
Perpustakaan       37
Sekretariat        56
Name: insGrp, dtype: int64

In [19]:
df_add_grp_inst = pd.concat([df.reset_index(drop=True),tmp[['insGrp']]], axis=1)

In [23]:
df_add_grp_inst.groupby('insGrp').agg(aggregat)

,jumlahFormasi,jumlahSubmit,jabNm
insGrp,,,
Arsip,71,2336,18
Kejaksaan,5203,78332,41
Kementerian,23564,1311490,12229
Kepolisian,554,4551,175
Lembaga,5211,120202,1892
Mahkamah,2104,58320,24
Ombudsman,91,1952,86
Pemerintah,113365,2616402,93636
Perpustakaan,57,2542,37


In [24]:
df_add_grp_inst.to_csv("cpns_2019_clean.csv", encoding='utf-8',index=False)